# Librerías

Importamos las librerías necesarias y cargamos variables de entorno

In [1]:
%pip install neo4j
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import sys
from pathlib import Path

ROOT = Path.cwd().parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

from apis.neo4j_api import Neo4jAPI
from apis.relationData import make_rel_id
from dotenv import load_dotenv
import os

load_dotenv(ROOT / ".env.local")

NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USER = os.getenv("NEO4J_USER")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

api = Neo4jAPI(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD)
# Reset inicial
api.clear_graph()

# Operaciones de Usuarios

Crear, actualizar, leer y eliminar usuarios

## Crear Usuarios

create_user(user_id, name, **props) crea un nodo usuario en NEO4J. El ID es único por lo que un ID duplicado lanza excepción.

In [3]:
try:
    api.create_user("u1", name="Ana")
    api.create_user("u2", name="Luis")
    api.create_user("u3", name="Marta")
    api.create_user("u4", name="Carlos")
    api.create_user("u5", name="Sofia")
except ValueError as e:
    print("Error al crear usuario:", e)
print()
# Intento de crear un usuario con ID duplicado
try:
    api.create_user("u1", name="Duplicate Ana")
except ValueError as e:
    print("Error: No se puede crear un usuario con ID duplicado:", e)
print()


Error: No se puede crear un usuario con ID duplicado: User 'u1' no se pudo crear o ya existe: {neo4j_code: Neo.ClientError.Schema.ConstraintValidationFailed} {message: Node(12) already exists with label `User` and property `user_id` = 'u1'} {gql_status: 50N42} {gql_status_description: error: general processing exception - unexpected error. Unexpected error has occurred. See debug log for details.}



## Actualizar Usuarios

update_user(user_id, **props) actualiza un nodo usuario en NEO4J. Actualiza todos los campos que vengan en props. Si el usuario no existe, devuelve None

In [4]:
u = api.get_user("u2")
print(u)

print("Actualizando usuario u2. Nuevo nombre: Luis Miguel")

u = api.update_user("u2", name="Luis Miguel")
print(u)

UserData(user_id='u2', name='Luis', props={'user_id': 'u2', 'name': 'Luis'})
Actualizando usuario u2. Nuevo nombre: Luis Miguel
UserData(user_id='u2', name='Luis Miguel', props={'user_id': 'u2', 'name': 'Luis Miguel'})


## Leer Usuarios

get_user(user_id) obtiene un nodo usuario de NEO4J. Si el usuario no existe, devuelve None

list_users(limit) obtiene todos los usuarios hasta un limite

In [5]:
users = api.list_users()
for user in users:
    print(user)

UserData(user_id='u1', name='Ana', props={'user_id': 'u1', 'name': 'Ana'})
UserData(user_id='u2', name='Luis Miguel', props={'user_id': 'u2', 'name': 'Luis Miguel'})
UserData(user_id='u3', name='Marta', props={'user_id': 'u3', 'name': 'Marta'})
UserData(user_id='u4', name='Carlos', props={'user_id': 'u4', 'name': 'Carlos'})
UserData(user_id='u5', name='Sofia', props={'user_id': 'u5', 'name': 'Sofia'})


## Borrar Usuarios

delete_user(user_id) elimina un nodo usuario de NEO4J. Devuelve True si lo borro, False si no lo logro

In [6]:
u = api.create_user("u6", name="Elena")
print(u)

print("Eliminando usuario u6")
api.delete_user("u6")
print("No existe usario u6: ", api.get_user("u6"))

UserData(user_id='u6', name='Elena', props={'user_id': 'u6', 'name': 'Elena'})
Eliminando usuario u6
No existe usario u6:  None


# Operaciones de Relaciones

Crear, actualizar, leer, eliminar relaciones

## Crear Relaciones

create_relation(sender_id, receiver_id, rel_id, msg_count, **props) crea una relación entre dos nodos usuario. Las relaciones son unidireccionales. Si la relación ya existe, lanza excepción

In [7]:
try:
    api.create_relation("u1", "u2")
    api.create_relation("u1", "u3")
    api.create_relation("u2", "u4")
    api.create_relation("u3", "u5")
    api.create_relation("u4", "u5")
    api.create_relation("u5", "u1")
except ValueError as e:
    print("Error al crear relación:", e)
print()
# Intento de crear una relación duplicada
try:
    api.create_relation("u1", "u2")
except ValueError as e:
    print("Error: No se puede crear una relación duplicada:", e)
print()


Error: No se puede crear una relación duplicada: No se pudo crear relación u1->u2 o ya existe: {neo4j_code: Neo.ClientError.Schema.ConstraintValidationFailed} {message: Relationship(52) already exists with type `CAN_MESSAGE` and property `rel_id` = 'u1-u2'} {gql_status: 50N42} {gql_status_description: error: general processing exception - unexpected error. Unexpected error has occurred. See debug log for details.}



## Actualizar Relaciones

update_relation(rel_id, **props) actualiza una relación. Actualiza todos los campos que vengan en props. Si la relación no existe, devuelve None

In [8]:
rel_id = make_rel_id("u1", "u2")
relation = api.update_relation(rel_id, msg_count=5)
print(relation)

rel_id = make_rel_id("u1", "u5")
relation = api.update_relation(rel_id, msg_count=3)
print("La relación u1 -> u5 no existe:", relation)

RelationData(rel_id='u1-u2', from_user_id='u1', to_user_id='u2', msg_count=5, props={'rel_id': 'u1-u2', 'msg_count': 5})
La relación u1 -> u5 no existe: None


## Leer Relaciones
get_relation(rel_id) obtiene una relación. Si la relación no existe, devuelve None

In [9]:
rel_id = make_rel_id("u3", "u5")
relation = api.get_relation(rel_id)
print(relation)

rel_id = make_rel_id("u2", "u5")
relation = api.get_relation(rel_id)
print("Relación u2 -> u5 no existe: ", relation)

RelationData(rel_id='u3-u5', from_user_id='u3', to_user_id='u5', msg_count=0, props={'rel_id': 'u3-u5', 'msg_count': 0})
Relación u2 -> u5 no existe:  None


## Borrar Relaciones

delete_relation(rel_id) elimina relación CAN_MESSAGE por rel_id. Devuelve True si se eliminó, False si no.

In [10]:
rel_id = make_rel_id("u3", "u5")
was_deleted = api.delete_relation(rel_id)
print(f"Relación u3 -> u5 eliminada: {was_deleted}")

Relación u3 -> u5 eliminada: True


## Incrementar Mensajes
increment_message_count(rel_id) o increment_message_users(sender_id, receiver_id) incrementan en uno el número de mensajes de la relación. Si la relación no existe, lanza excepción

In [11]:
total_msgs = api.increment_by_users("u1", "u2")
print(f"Total mensajes de u1 a u2 después de incremento: {total_msgs}")

Total mensajes de u1 a u2 después de incremento: 6


# Exportar e Importar (Operaciones de Snapshot)

# Exportar nodos
export_nodes() exporta todo los nodos usuario actuales. Devuelve una lista de diccionarios (nodos)

In [12]:
nodes = api.export_nodes()
for node in nodes:
    print(node)

{'user_id': 'u1', 'props': {'user_id': 'u1', 'name': 'Ana'}}
{'user_id': 'u2', 'props': {'user_id': 'u2', 'name': 'Luis Miguel'}}
{'user_id': 'u3', 'props': {'user_id': 'u3', 'name': 'Marta'}}
{'user_id': 'u4', 'props': {'user_id': 'u4', 'name': 'Carlos'}}
{'user_id': 'u5', 'props': {'user_id': 'u5', 'name': 'Sofia'}}


# Exportar Relaciones
export_relationships() exporta todas las relaciones CAN_MESSAGE actuales. Devuelve una lista de diccionarios (relaciones)

In [13]:
relations = api.export_relationships()
for relation in relations:
    print(relation)

{'from_id': 'u1', 'to_id': 'u2', 'rel_id': 'u1-u2', 'props': {'rel_id': 'u1-u2', 'msg_count': 6}}
{'from_id': 'u1', 'to_id': 'u3', 'rel_id': 'u1-u3', 'props': {'rel_id': 'u1-u3', 'msg_count': 0}}
{'from_id': 'u2', 'to_id': 'u4', 'rel_id': 'u2-u4', 'props': {'rel_id': 'u2-u4', 'msg_count': 0}}
{'from_id': 'u4', 'to_id': 'u5', 'rel_id': 'u4-u5', 'props': {'rel_id': 'u4-u5', 'msg_count': 0}}
{'from_id': 'u5', 'to_id': 'u1', 'rel_id': 'u5-u1', 'props': {'rel_id': 'u5-u1', 'msg_count': 0}}


# Importar Nodos y Relaciones
import_nodes(nodes) importa los nodos que le pasan por parametros. Si ya existe el nodo, lo sobrescribe (MERGE)

import_relationships(relationships) importa las relaciones que le pasan por parametros. Si ya existe la relación, la sobrescribe (MERGE)

In [14]:
api.clear_graph()
api.import_nodes(nodes)
api.import_relationships(relations)
print(api.list_users())
print(api.get_relation(make_rel_id("u1", "u2")))

[UserData(user_id='u1', name='Ana', props={'user_id': 'u1', 'name': 'Ana'}), UserData(user_id='u2', name='Luis Miguel', props={'user_id': 'u2', 'name': 'Luis Miguel'}), UserData(user_id='u3', name='Marta', props={'user_id': 'u3', 'name': 'Marta'}), UserData(user_id='u4', name='Carlos', props={'user_id': 'u4', 'name': 'Carlos'}), UserData(user_id='u5', name='Sofia', props={'user_id': 'u5', 'name': 'Sofia'})]
RelationData(rel_id='u1-u2', from_user_id='u1', to_user_id='u2', msg_count=6, props={'rel_id': 'u1-u2', 'msg_count': 6})


# Consultas

# Vecinos
neighbors(user_id, limit) devuelve todos los vecinos conectados al user_id pasado por parametros mediante la relación CAN_MESSAGE

In [15]:
neighbors = api.neighbors("u1")
for neighbor in neighbors:
    print(neighbor)

u2
u3
u5


# Vecinos Ordenados por Número de Mensajes
neighbors_ordered_by_messages(user_id, limit) devuelve todos los vecinos conectados al user_id pasado por parametros mediante la relación CAN_MESSAGE ordenados por número de mensajes

In [16]:
neighbors = api.neighbors_ordered_by_messages("u1")
for neighbor in neighbors:
    print(neighbor)

('u2', 6)
('u3', 0)
('u5', 0)


# Camino más corto ponderado por número de mensajes
best_path_by_messages(src_user_id, dst_usr_id) devuelve el camino más corto por número de mensajes entre dos ndos. Se aplica la relación **1/numero_de_mensajes**

In [17]:
path = api.best_path_by_messages("u1", "u3")
print(path)

{'nodes': ['u1', 'u3'], 'hops': 1, 'cost': 1.0}


In [18]:
api.close()